In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Value Backtester Notebook\n",
    "\n",
    "This notebook replicates the functionality of `app.py` but in a sequential workflow for experiments."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import yfinance as yf\n",
    "from datetime import datetime\n",
    "\n",
    "from src.backtester import Backtester\n",
    "from src.valuation import score_valuations"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Parameters"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "tickers_input = \"AAPL,MSFT,AMD\"\n",
    "start = pd.to_datetime(\"2018-01-01\")\n",
    "end = pd.to_datetime(datetime.today().date())\n",
    "rebal_freq = \"Monthly\"  # or Quarterly\n",
    "top_n = 5\n",
    "\n",
    "risk_free = 0.04\n",
    "discount_rate = 0.08\n",
    "terminal_growth = 0.02\n",
    "projection_years = 5\n",
    "peer_pes = \"AAPL:25,MSFT:30,AMD:40\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Load Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "def load_yfinance(tickers, start, end):\n",
    "    tickers = [t.strip().upper() for t in tickers.split(\",\") if t.strip() != \"\"]\n",
    "    data = {}\n",
    "    for t in tickers:\n",
    "        df = yf.download(t, start=start, end=end, progress=False)\n",
    "        if df.empty:\n",
    "            continue\n",
    "        if isinstance(df.columns, pd.MultiIndex):\n",
    "            if (\"Adj Close\", t) in df.columns:\n",
    "                series = df[(\"Adj Close\", t)]\n",
    "            else:\n",
    "                adj_cols = [col for col in df.columns if col[0] == \"Adj Close\"]\n",
    "                if adj_cols:\n",
    "                    series = df[adj_cols[0]]\n",
    "                else:\n",
    "                    continue\n",
    "        else:\n",
    "            if \"Adj Close\" not in df.columns:\n",
    "                continue\n",
    "            series = df[\"Adj Close\"]\n",
    "        data[t] = series.rename(t)\n",
    "    if not data:\n",
    "        return pd.DataFrame()\n",
    "    price_df = pd.concat(data.values(), axis=1)\n",
    "    price_df.index = pd.to_datetime(price_df.index)\n",
    "    return price_df\n",
    "\n",
    "price_df = load_yfinance(tickers_input, start, end)\n",
    "price_df.tail()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Valuation Scoring"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "valuation_scores = score_valuations(\n",
    "    price_df.columns.tolist(),\n",
    "    price_df.iloc[-1].to_dict(),\n",
    "    discount_rate,\n",
    "    terminal_growth,\n",
    "    projection_years,\n",
    "    peer_pes\n",
    ")\n",
    "\n",
    "pd.DataFrame(valuation_scores).T.rename(columns={0: \"score\", 1: \"dcf_value\", 2: \"comparables_pe_est\"}).sort_values(\"score\").head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Backtest"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "freq = \"M\" if rebal_freq == \"Monthly\" else \"Q\"\n",
    "bt = Backtester(price_df)\n",
    "portfolio, stats = bt.run_strategy(\n",
    "    valuation_scores,\n",
    "    top_n=top_n,\n",
    "    rebalance=freq,\n",
    "    risk_free_annual=risk_free\n",
    ")\n",
    "stats"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Plots"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, ax = plt.subplots(figsize=(10, 5))\n",
    "portfolio['cum_portfolio'].plot(ax=ax, label=\"Strategy\")\n",
    "portfolio['cum_benchmark'].plot(ax=ax, label=\"Benchmark (equal weight)\")\n",
    "ax.legend()\n",
    "ax.set_title(\"Cumulative returns\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig2, ax2 = plt.subplots(figsize=(10, 3))\n",
    "portfolio['portfolio_drawdown'].plot(ax=ax2)\n",
    "ax2.set_title(\"Portfolio drawdown\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "portfolio[['holdings']].tail(10)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Value Backtester Notebook\n',
    '\n',
    'This notebook replicates the functionality of `app.py` but in a sequential workflow for experiments.']},
  {'cell_type': 'code',
   'execution_count': 1,
   'metadata': {},
   'outputs': [],
   'source': ['import pandas as pd\n',
    'import numpy as np\n',
    'import matplotlib.pyplot as plt\n',
    'import yfinance as yf\n',
    'from datetime import datetime\n',
    '\n',
    'from src.backtester import Backtester\n',
    'from src.valuation import score_valuations']},
  {'cell_type': 'markdown', 'metadata': {}, 'source': ['## Parameters']},
  {'cell_type': 'code',
   'execution_count': 2,
   'metadata': {},
   'outputs': [],
   'source': ['tickers_input = "AAPL,MSFT,AMD"\n',
    'start = pd.to_datetime("2018-01-01")\n',
    'end = pd.to_datetime(datetime.today().date())\n',
    'rebal_freq = "Monthly"  # or Quarterly\n',
    'top_n = 5\n',
    '\n',
    'risk_free 